In [ ]:
#First Stage of automating exception reports

In [ ]:
#Downloading the raw data from Amazon Web Service (AWS) S3 
#This is the cloud warehouse where we store data for use on the analytical platform

#This is a package that allows us to download data from AWS S3 in a more usable way
#You may need to install it in the Terminal --> pip install arrow-pd-parser
from arrow_pd_parser import reader, writer

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Specifying the reader Both reader statements are equivalent and call the same readers under the hood
#Location of data - setting up the folder and file names
bucket = "alpha-piatool/"
folder = "sopdq_AV/"
raw_data = "WFPT Staff Data Mar22.csv"
filename = bucket+folder+raw_data

In [ ]:
#Loading the raw data stored by David Yuen 
raw_data = reader.read("s3://"+filename, file_format="csv")

In [ ]:
raw_data.head(2)

In [ ]:
#Tracie Kilbey creates four sheets from the raw dataset, which she then uses for the exception reports
#Let's start with creating the first sheet, as it's the simplest - "WfPT all"
#This is just the raw data, but we'll make a few changes to column headings to make it easier to use

In [ ]:
WfPT_all = raw_data

In [ ]:
WfPT_all.columns = WfPT_all.columns.str.replace(" ", "_")
WfPT_all.columns = map(str.lower, WfPT_all.columns)

In [ ]:
#Completed first file
WfPT_all.head(3)

In [ ]:
# Now create second file - Probation inc HQ
# Filter Column "NOMS-MOJ" to leave only "NPS" or "NOMS HQ"

In [ ]:
Probation_inc_HQ = raw_data[raw_data["noms-moj"].isin(["NOMS HQ", "NPS"])]

In [ ]:
#Completed second file
Probation_inc_HQ.head(2)

In [ ]:
# Now create the third file - Probation some HQ
# 